In [0]:
!pip install --upgrade google-cloud-videointelligence

In [0]:
from google.colab import files
files.upload()

In [0]:
!ls

In [0]:
from google.cloud import videointelligence
import os, io

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'service-account-video.json'

In [0]:
path = r'gs://cloudmleap/video/next/JaneGoodall.mp4'

# Challenge : Tracking objects

In [0]:
def starter(path):

    video_client = videointelligence.VideoIntelligenceServiceClient()
    # Determine the correct feature to use
    features = [videointelligence.enums.Feature ...]
    
    if path.startswith('gs:'):
        operation = video_client.annotate_video(input_uri=path, features=features)     
    else:
      with io.open(path, 'rb') as video_file:
          input_content = video_file.read()
      operation = video_client.annotate_video(features=features, input_content=input_content)
    
    print('\nProcessing video for object annotations.')

    result = operation.result(timeout=300)
    print('\nFinished processing.\n')
 
    return result  



In [0]:
result = starter(path)
result

In [0]:
# Determine how to parse the output from the API so it can be used below
object_annotations = ...

In [0]:
for object_annotation in object_annotations:
    print('Entity description: {}'.format(
        object_annotation.entity.description))
    if object_annotation.entity.entity_id:
        print('Entity id: {}'.format(object_annotation.entity.entity_id))

    print('Segment: {}s to {}s'.format(
        object_annotation.segment.start_time_offset.seconds +
        object_annotation.segment.start_time_offset.nanos / 1e9,
        object_annotation.segment.end_time_offset.seconds +
        object_annotation.segment.end_time_offset.nanos / 1e9))

    print('Confidence: {}'.format(object_annotation.confidence))

    # Print only bounding box of the first frame in the segment
    frame = object_annotation.frames[0]
    box = frame.normalized_bounding_box
    print('Time offset of the first frame: {}s'.format(
        frame.time_offset.seconds + frame.time_offset.nanos / 1e9))
    print('Bounding box position:')
    print('\tleft  : {}'.format(box.left))
    print('\ttop   : {}'.format(box.top))
    print('\tright : {}'.format(box.right))
    print('\tbottom: {}'.format(box.bottom))
    print('\n')

# Challenge : Transcribing videos

In [0]:
from google.cloud import videointelligence
from google.cloud.videointelligence import types
import os, io

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'service-account-video.json'

def starter(path):

    video_client = videointelligence.VideoIntelligenceServiceClient()
    # Determine the correct feature to use
    features = [videointelligence.enums.Feature ...]
 
    if path.startswith('gs:'):
      config = videointelligence.types.SpeechTranscriptionConfig(language_code='en-US',enable_automatic_punctuation=True)
      context = videointelligence.types.VideoContext(speech_transcription_config=config)
      operation = video_client.annotate_video(path, features=features, video_context=context)
        
    else:
      with io.open(path, 'rb') as video_file:
          input_content = video_file.read()
      config = videointelligence.types.SpeechTranscriptionConfig(language_code='en-US',enable_automatic_punctuation=True)
      context = videointelligence.types.VideoContext(speech_transcription_config=config)
      operation = video_client.annotate_video(features=features, input_content=input_content, video_context=context)
    
    print('\nProcessing video for speech transcription:')
    result = operation.result(timeout=600)
    print('\nProcessing complete.\n')
 
    return result  